In [ ]:
import pip

In [27]:
pip.main(['install','/home/nitesh/fastText/.'])

  Using cached https://files.pythonhosted.org/packages/7b/61/11b05cc37ccdaabad89f04dbdc2a02905cf6de6f9b05816dba843beed328/numpy-1.14.3-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8c/10/79282747f9169f21c053c562a0baa21815a8c7879be97abd930dbcf862e8/setuptools-39.1.0-py2.py3-none-any.whl


You are using pip version 8.1.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


0

In [1]:
import numpy as np
import pip
import torch
from torch import nn
import fastText
import argparse
from torch.autograd import Variable
import io

In [2]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--dis_hidden_dim', type=int, default=2048, help='dimension of hidden state')
# parser.add_argument('--dis_dropout',type=float, default=0.5, help='dropout value')
# parser.add_argument('--dis_input_dropout', type=float, default=0.3, help='Point Number [default: 1024]')

# params=parser.parse_args()
def load_vec(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

def load_fasttext_model(path):
    """
    Load a binarized fastText model.
    """
    try:
        import fastText
    except ImportError:
        raise Exception("Unable to import fastText. Please install fastText for Python: "
                        "https://github.com/facebookresearch/fastText")
    return fastText.load_model(path)

In [3]:
# english.get
# src_embeddings, src_id2word, src_word2id = load_vec(src_path, nmax)
# tgt_embeddings, tgt_id2word, tgt_word2id = load_vec(tgt_path, nmax)

In [4]:

english=load_fasttext_model("/home/nitesh/wordembeddings/data/wiki.en.bin")
spanish=load_fasttext_model("/home/nitesh/wordembeddings/data/wiki.es.bin")
words = english.get_labels()
english_embeddings = torch.from_numpy(np.concatenate([english.get_word_vector(w)[None] for w in words], 0))
Swords = spanish.get_labels()
spanish_embeddings = torch.from_numpy(np.concatenate([spanish.get_word_vector(w)[None] for w in Swords], 0))

In [5]:
def get_batch2(X,M):
    '''
    get batch for pytorch model
    '''
    # TODO fix and make it nicer, there is pytorch forum question
    #X,Y = X.data.numpy(), Y.data.numpy()
#     X,Y = X, Y
    N = X.size()[0]
    N=200000
    batch_indices = torch.LongTensor( np.random.randint(0,N,size=M) )
#     print(batch_indices)
    batch_xs = torch.index_select(X,0,batch_indices)
#     batch_ys = torch.index_select(Y,0,batch_indices)
    return Variable(batch_xs, requires_grad=False).cuda()

In [6]:
class Discriminator(nn.Module):

    def __init__(self, dis_hid_dim,dis_dropout,dis_input_dropout):
        super(Discriminator, self).__init__()

        self.emb_dim = 300
        self.dis_layers = 3
        self.dis_hid_dim = dis_hid_dim
        self.dis_dropout = dis_dropout
        self.dis_input_dropout = dis_input_dropout

        layers = [nn.Dropout(self.dis_input_dropout)]
        for i in range(self.dis_layers + 1):
            input_dim = self.emb_dim if i == 0 else self.dis_hid_dim
            output_dim = 1 if i == self.dis_layers else self.dis_hid_dim
            layers.append(nn.Linear(input_dim, output_dim))
            if i < self.dis_layers:
                layers.append(nn.LeakyReLU(0.2))
                layers.append(nn.Dropout(self.dis_dropout))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        assert x.dim() == 2 and x.size(1) == self.emb_dim
        return self.layers(x).view(-1)

In [7]:

def Orthogonal(mapping,beta):
    W=mapping.weight.data
#     print(type(W))
  
    mapping.weight.data.copy_((1 + beta) * W - beta * W.mm(W.transpose(0, 1).mm(W)))
#     print(mapping.weight.data.transpose(0, 1).mm(W))
    
    return mapping


In [11]:
#set up linear mapping,
mapping = nn.Linear(300, 300, bias=False)
mapping.weight.data.copy_(torch.eye(300))
modelDisc= Discriminator(2048,0.3,0.1)
mapping.cuda()
modelDisc.cuda()

Discriminator(
  (layers): Sequential(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=300, out_features=2048, bias=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Dropout(p=0.3)
    (4): Linear(in_features=2048, out_features=2048, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Dropout(p=0.3)
    (7): Linear(in_features=2048, out_features=2048, bias=True)
    (8): LeakyReLU(negative_slope=0.2)
    (9): Dropout(p=0.3)
    (10): Linear(in_features=2048, out_features=1, bias=True)
    (11): Sigmoid()
  )
)

In [12]:
linoptimizer = torch.optim.SGD(mapping.parameters(), lr=0.1)
discoptimizer= torch.optim.SGD(modelDisc.parameters(), lr=0.1)
# torch.optim.
loss_fn=torch.nn.BCELoss()

In [13]:
epoch=10000
batch_size=16
# E_batch=Batch(english_embeddings)
# S_batch=Batch(spanish_embeddings)
dog=Variable(english_embeddings[english.get_word_id('the')]).cuda()

perro=spanish_embeddings[spanish.get_word_id('el')]
for _ in range(epoch):
      
    
    modelDisc.train()
    mapping.eval()
    for __ in range(5):
        
        #get minibatch of spanish words
        spanish_batch=get_batch2(spanish_embeddings,batch_size)
        #getminibatch of english words
        english_batch=get_batch2(english_embeddings,batch_size)
        
        #generate fake spanish embeddings
        s_fake= mapping(english_batch)
        
        #stack real and fake
        x = torch.cat([spanish_batch, s_fake], 0)
#         print(x.shape)
        y = torch.FloatTensor(2 * batch_size).zero_().cuda()
        
        y[batch_size:] = 0.1
        y[:batch_size]=1-0.1
        y_pred = modelDisc(x)
#         print(y_pred.shape)
        # Compute and print loss.
        

        loss= loss_fn(y_pred, y)
#         print(y_pred[0])
        discoptimizer.zero_grad()
       
        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()
#         print(loss.data)

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        discoptimizer.step()
        
        
    for __ in range(1):
        modelDisc.eval()
        mapping.train()
        spanish_batch=get_batch2(spanish_embeddings,batch_size)
        #getminibatch of english words
        english_batch=get_batch2(english_embeddings,batch_size)
        
        #generate fake spanish embeddings
        s_fake= mapping(english_batch)
        
        #stack real and fake
        x = torch.cat([spanish_batch, s_fake], 0)
#         print(x.shape)
        y = torch.FloatTensor(2 * batch_size).zero_().cuda()
        
        y[batch_size:] = 1-0.1
        y[:batch_size]=0.1
        y_pred = modelDisc(x)
        loss2= loss_fn(y_pred, y)
        linoptimizer.zero_grad()
        loss2.backward()
        linoptimizer.step()
        
        if _%1:
            mapping=Orthogonal(mapping,0.001)
        cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    #     print(type(perro),type(mapping(dog).cpu()))
    print(cos(perro.cpu(),mapping(dog).cpu()))
    
    
    

tensor(1.00000e-04 *
       8.8838)
tensor(1.00000e-03 *
       1.0724)
tensor(1.00000e-03 *
       1.4233)
tensor(1.00000e-03 *
       1.8968)
tensor(1.00000e-03 *
       2.5858)
tensor(1.00000e-03 *
       3.4653)
tensor(1.00000e-03 *
       5.1544)
tensor(1.00000e-03 *
       7.6525)
tensor(1.00000e-02 *
       1.1111)
tensor(1.00000e-02 *
       1.6401)
tensor(1.00000e-02 *
       2.3840)
tensor(1.00000e-02 *
       3.6189)
tensor(1.00000e-02 *
       5.0643)
tensor(1.00000e-02 *
       6.8286)
tensor(1.00000e-02 *
       8.7569)
tensor(0.1120)
tensor(0.1401)
tensor(0.1709)
tensor(0.1944)
tensor(0.2124)
tensor(0.2251)
tensor(0.2362)
tensor(0.2463)
tensor(0.2530)
tensor(0.2604)
tensor(0.2651)
tensor(0.2692)
tensor(0.2728)
tensor(0.2751)
tensor(0.2767)
tensor(0.2776)
tensor(0.2779)
tensor(0.2775)
tensor(0.2767)
tensor(0.2755)
tensor(0.2739)
tensor(0.2722)
tensor(0.2700)
tensor(0.2675)
tensor(0.2652)
tensor(0.2627)
tensor(0.2600)
tensor(0.2567)
tensor(0.2537)
tensor(0.2501)
tensor(0.2

tensor(0.3150)
tensor(0.3144)
tensor(0.3146)
tensor(0.3158)
tensor(0.3200)
tensor(0.3225)
tensor(0.3275)
tensor(0.3310)
tensor(0.3334)
tensor(0.3341)
tensor(0.3331)
tensor(0.3337)
tensor(0.3313)
tensor(0.3310)
tensor(0.3301)
tensor(0.3311)
tensor(0.3325)
tensor(0.3338)
tensor(0.3329)
tensor(0.3318)
tensor(0.3289)
tensor(0.3283)
tensor(0.3273)
tensor(0.3264)
tensor(0.3275)
tensor(0.3295)
tensor(0.3297)
tensor(0.3277)
tensor(0.3273)
tensor(0.3269)
tensor(0.3267)
tensor(0.3294)
tensor(0.3322)
tensor(0.3346)
tensor(0.3342)
tensor(0.3345)
tensor(0.3321)
tensor(0.3281)
tensor(0.3253)
tensor(0.3232)
tensor(0.3204)
tensor(0.3189)
tensor(0.3185)
tensor(0.3193)
tensor(0.3222)
tensor(0.3253)
tensor(0.3289)
tensor(0.3336)
tensor(0.3355)
tensor(0.3380)
tensor(0.3392)
tensor(0.3399)
tensor(0.3402)
tensor(0.3391)
tensor(0.3364)
tensor(0.3352)
tensor(0.3355)
tensor(0.3353)
tensor(0.3352)
tensor(0.3312)
tensor(0.3273)
tensor(0.3232)
tensor(0.3210)
tensor(0.3208)
tensor(0.3215)
tensor(0.3214)
tensor(0.3

tensor(0.3480)
tensor(0.3443)
tensor(0.3448)
tensor(0.3455)
tensor(0.3447)
tensor(0.3439)
tensor(0.3435)
tensor(0.3445)
tensor(0.3465)
tensor(0.3500)
tensor(0.3520)
tensor(0.3565)
tensor(0.3600)
tensor(0.3611)
tensor(0.3603)
tensor(0.3620)
tensor(0.3601)
tensor(0.3607)
tensor(0.3586)
tensor(0.3552)
tensor(0.3556)
tensor(0.3515)
tensor(0.3504)
tensor(0.3504)
tensor(0.3478)
tensor(0.3501)
tensor(0.3481)
tensor(0.3502)
tensor(0.3502)
tensor(0.3506)
tensor(0.3478)
tensor(0.3506)
tensor(0.3519)
tensor(0.3543)
tensor(0.3520)
tensor(0.3495)
tensor(0.3507)
tensor(0.3537)
tensor(0.3513)
tensor(0.3521)
tensor(0.3492)
tensor(0.3489)
tensor(0.3474)
tensor(0.3462)
tensor(0.3482)
tensor(0.3484)
tensor(0.3495)
tensor(0.3466)
tensor(0.3463)
tensor(0.3495)
tensor(0.3538)
tensor(0.3576)
tensor(0.3599)
tensor(0.3617)
tensor(0.3613)
tensor(0.3606)
tensor(0.3568)
tensor(0.3539)
tensor(0.3504)
tensor(0.3479)
tensor(0.3472)
tensor(0.3511)
tensor(0.3546)
tensor(0.3560)
tensor(0.3591)
tensor(0.3631)
tensor(0.3

tensor(0.3397)
tensor(0.3380)
tensor(0.3381)
tensor(0.3395)
tensor(0.3401)
tensor(0.3384)
tensor(0.3379)
tensor(0.3358)
tensor(0.3314)
tensor(0.3254)
tensor(0.3182)
tensor(0.3136)
tensor(0.3121)
tensor(0.3109)
tensor(0.3109)
tensor(0.3118)
tensor(0.3122)
tensor(0.3146)
tensor(0.3156)
tensor(0.3197)
tensor(0.3243)
tensor(0.3285)
tensor(0.3277)
tensor(0.3268)
tensor(0.3293)
tensor(0.3300)
tensor(0.3309)
tensor(0.3262)
tensor(0.3266)
tensor(0.3268)
tensor(0.3245)
tensor(0.3241)
tensor(0.3205)
tensor(0.3201)
tensor(0.3205)
tensor(0.3190)
tensor(0.3194)
tensor(0.3208)
tensor(0.3226)
tensor(0.3227)
tensor(0.3245)
tensor(0.3232)
tensor(0.3227)
tensor(0.3229)
tensor(0.3216)
tensor(0.3242)
tensor(0.3245)
tensor(0.3291)
tensor(0.3288)
tensor(0.3290)
tensor(0.3291)
tensor(0.3308)
tensor(0.3321)
tensor(0.3319)
tensor(0.3344)
tensor(0.3373)
tensor(0.3357)
tensor(0.3343)
tensor(0.3365)
tensor(0.3362)
tensor(0.3367)
tensor(0.3366)
tensor(0.3333)
tensor(0.3288)
tensor(0.3274)
tensor(0.3242)
tensor(0.3

tensor(0.3186)
tensor(0.3179)
tensor(0.3174)
tensor(0.3197)
tensor(0.3198)
tensor(0.3196)
tensor(0.3174)
tensor(0.3195)
tensor(0.3195)
tensor(0.3180)
tensor(0.3177)
tensor(0.3179)
tensor(0.3202)
tensor(0.3228)
tensor(0.3267)
tensor(0.3267)
tensor(0.3272)
tensor(0.3274)
tensor(0.3319)
tensor(0.3302)
tensor(0.3283)
tensor(0.3257)
tensor(0.3214)
tensor(0.3218)
tensor(0.3192)
tensor(0.3165)
tensor(0.3199)
tensor(0.3228)
tensor(0.3275)
tensor(0.3302)
tensor(0.3301)
tensor(0.3349)
tensor(0.3362)
tensor(0.3378)
tensor(0.3376)
tensor(0.3380)
tensor(0.3362)
tensor(0.3361)
tensor(0.3319)
tensor(0.3265)
tensor(0.3269)
tensor(0.3255)
tensor(0.3279)
tensor(0.3256)
tensor(0.3263)
tensor(0.3265)
tensor(0.3279)
tensor(0.3280)
tensor(0.3276)
tensor(0.3289)
tensor(0.3284)
tensor(0.3302)
tensor(0.3298)
tensor(0.3313)
tensor(0.3280)
tensor(0.3270)
tensor(0.3247)
tensor(0.3227)
tensor(0.3199)
tensor(0.3192)
tensor(0.3172)
tensor(0.3173)
tensor(0.3210)
tensor(0.3214)
tensor(0.3214)
tensor(0.3237)
tensor(0.3

...

tensor(0.3668)
tensor(0.3671)
tensor(0.3664)
tensor(0.3659)
tensor(0.3660)
tensor(0.3667)
tensor(0.3672)
tensor(0.3683)
tensor(0.3668)
tensor(0.3657)
tensor(0.3641)
tensor(0.3634)
tensor(0.3638)
tensor(0.3708)
tensor(0.3718)
tensor(0.3767)
tensor(0.3756)
tensor(0.3747)
tensor(0.3718)
tensor(0.3684)
tensor(0.3680)
tensor(0.3664)
tensor(0.3653)
tensor(0.3674)
tensor(0.3634)
tensor(0.3603)
tensor(0.3609)
tensor(0.3597)
tensor(0.3581)
tensor(0.3569)
tensor(0.3570)
tensor(0.3615)
tensor(0.3633)
tensor(0.3641)
tensor(0.3698)
tensor(0.3718)
tensor(0.3718)
tensor(0.3697)
tensor(0.3666)
tensor(0.3668)
tensor(0.3624)
tensor(0.3569)
tensor(0.3580)
tensor(0.3587)
tensor(0.3595)
tensor(0.3616)
tensor(0.3630)
tensor(0.3634)
tensor(0.3695)
tensor(0.3714)
tensor(0.3713)
tensor(0.3735)
tensor(0.3736)
tensor(0.3748)
tensor(0.3753)
tensor(0.3747)
tensor(0.3761)
tensor(0.3753)
tensor(0.3684)
tensor(0.3652)
tensor(0.3647)
tensor(0.3585)
tensor(0.3533)
tensor(0.3524)
tensor(0.3535)
tensor(0.3527)
tensor(0.3

tensor(0.3636)
tensor(0.3641)
tensor(0.3604)
tensor(0.3580)
tensor(0.3507)
tensor(0.3494)
tensor(0.3441)
tensor(0.3395)
tensor(0.3402)
tensor(0.3415)
tensor(0.3416)
tensor(0.3395)
tensor(0.3422)
tensor(0.3418)
tensor(0.3453)
tensor(0.3510)
tensor(0.3530)
tensor(0.3566)
tensor(0.3547)
tensor(0.3583)
tensor(0.3620)
tensor(0.3608)
tensor(0.3601)
tensor(0.3605)
tensor(0.3631)
tensor(0.3676)
tensor(0.3724)
tensor(0.3757)
tensor(0.3731)
tensor(0.3684)
tensor(0.3661)
tensor(0.3658)
tensor(0.3603)
tensor(0.3593)
tensor(0.3546)
tensor(0.3541)
tensor(0.3511)
tensor(0.3569)
tensor(0.3566)
tensor(0.3571)
tensor(0.3587)
tensor(0.3626)
tensor(0.3629)
tensor(0.3617)
tensor(0.3655)
tensor(0.3682)
tensor(0.3663)
tensor(0.3662)
tensor(0.3653)
tensor(0.3694)
tensor(0.3705)
tensor(0.3711)
tensor(0.3696)
tensor(0.3702)
tensor(0.3700)
tensor(0.3692)
tensor(0.3663)
tensor(0.3617)
tensor(0.3579)
tensor(0.3553)
tensor(0.3535)
tensor(0.3509)
tensor(0.3502)
tensor(0.3517)
tensor(0.3511)
tensor(0.3491)
tensor(0.3

tensor(0.3826)
tensor(0.3804)
tensor(0.3838)
tensor(0.3865)
tensor(0.3903)
tensor(0.3906)
tensor(0.3899)
tensor(0.3903)
tensor(0.3850)
tensor(0.3806)
tensor(0.3823)
tensor(0.3783)
tensor(0.3797)
tensor(0.3789)
tensor(0.3760)
tensor(0.3739)
tensor(0.3730)
tensor(0.3730)
tensor(0.3718)
tensor(0.3707)
tensor(0.3731)
tensor(0.3696)
tensor(0.3665)
tensor(0.3622)
tensor(0.3640)
tensor(0.3694)
tensor(0.3656)
tensor(0.3659)
tensor(0.3658)
tensor(0.3653)
tensor(0.3663)
tensor(0.3699)
tensor(0.3681)
tensor(0.3673)
tensor(0.3721)
tensor(0.3717)
tensor(0.3700)
tensor(0.3727)
tensor(0.3735)
tensor(0.3723)
tensor(0.3698)
tensor(0.3696)
tensor(0.3738)
tensor(0.3747)
tensor(0.3783)
tensor(0.3834)
tensor(0.3873)
tensor(0.3871)
tensor(0.3866)
tensor(0.3896)
tensor(0.3902)
tensor(0.3897)
tensor(0.3896)
tensor(0.3877)
tensor(0.3851)
tensor(0.3884)
tensor(0.3902)
tensor(0.3889)
tensor(0.3891)
tensor(0.3848)
tensor(0.3814)
tensor(0.3790)
tensor(0.3778)
tensor(0.3719)
tensor(0.3709)
tensor(0.3691)
tensor(0.3

tensor(0.3591)
tensor(0.3573)
tensor(0.3567)
tensor(0.3596)
tensor(0.3599)
tensor(0.3605)
tensor(0.3579)
tensor(0.3601)
tensor(0.3608)
tensor(0.3636)
tensor(0.3659)
tensor(0.3673)
tensor(0.3675)
tensor(0.3705)
tensor(0.3740)
tensor(0.3795)
tensor(0.3830)
tensor(0.3830)
tensor(0.3826)
tensor(0.3831)
tensor(0.3789)
tensor(0.3784)
tensor(0.3812)
tensor(0.3791)
tensor(0.3795)
tensor(0.3799)
tensor(0.3810)
tensor(0.3825)
tensor(0.3809)
tensor(0.3819)
tensor(0.3817)
tensor(0.3798)
tensor(0.3785)
tensor(0.3791)
tensor(0.3780)
tensor(0.3804)
tensor(0.3805)
tensor(0.3817)
tensor(0.3806)
tensor(0.3789)
tensor(0.3781)
tensor(0.3769)
tensor(0.3776)
tensor(0.3754)
tensor(0.3772)
tensor(0.3780)
tensor(0.3760)
tensor(0.3722)
tensor(0.3711)
tensor(0.3679)
tensor(0.3670)
tensor(0.3647)
tensor(0.3653)
tensor(0.3675)
tensor(0.3698)
tensor(0.3786)
tensor(0.3810)
tensor(0.3842)
tensor(0.3820)
tensor(0.3795)
tensor(0.3823)
tensor(0.3814)
tensor(0.3842)
tensor(0.3845)
tensor(0.3853)
tensor(0.3847)
tensor(0.3

tensor(0.3952)
tensor(0.3951)
tensor(0.4022)
tensor(0.4080)
tensor(0.4150)
tensor(0.4155)
tensor(0.4124)
tensor(0.4119)
tensor(0.4101)
tensor(0.4106)
tensor(0.4092)
tensor(0.4102)
tensor(0.4054)
tensor(0.4025)
tensor(0.4034)
tensor(0.4002)
tensor(0.3981)
tensor(0.3946)
tensor(0.3914)
tensor(0.3886)
tensor(0.3890)
tensor(0.3882)
tensor(0.3866)
tensor(0.3888)
tensor(0.3889)
tensor(0.3882)
tensor(0.3878)
tensor(0.3869)
tensor(0.3859)
tensor(0.3862)
tensor(0.3859)
tensor(0.3880)
tensor(0.3888)
tensor(0.3931)
tensor(0.3995)
tensor(0.3950)
tensor(0.3971)
tensor(0.3943)
tensor(0.3934)
tensor(0.3901)
tensor(0.3933)
tensor(0.3907)
tensor(0.3902)
tensor(0.3920)
tensor(0.3889)
tensor(0.3893)
tensor(0.3895)
tensor(0.3868)
tensor(0.3843)
tensor(0.3832)
tensor(0.3844)
tensor(0.3841)
tensor(0.3856)
tensor(0.3919)
tensor(0.3862)
tensor(0.3876)
tensor(0.3872)
tensor(0.3810)
tensor(0.3812)


In [14]:
# spanish_embeddings
tgt_emd=spanish_embeddings.numpy()
# print(tgt_emd.shape)


def get_nn(word_emb, tgt_emb, tgt_id2word, K=5):
  
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [16]:
def create_map(words,language):
    id2word={}
    for word in words:
        id=language.get_word_id(word)
        id2word[id]=word
    return id2word
spanish_map=create_map(Swords,spanish)
        

In [19]:
cat_embedding=mapping(Variable(english_embeddings[english.get_word_id('dog')],requires_grad=False).cuda()).cpu()
word_emd=cat_embedding.detach().numpy()
get_nn(word_emd,tgt_emd,spanish_map)


0.5354 - yrala
0.5353 - marilinda
0.5274 - berdaxagar
0.5259 - maripepa
0.5251 - anjela


In [23]:
src_embeddings=mapping(Variable(english_embeddings, requires_grad=False).cuda())


RuntimeError: cuda runtime error (2) : out of memory at /pytorch/aten/src/THC/generic/THCStorage.cu:58

In [25]:
src_embedding=src_embeddings.cpu().detach().numpy()
from sklearn.decomposition import PCA
pca = PCA(n_components=2, whiten=True)  # TSNE(n_components=2, n_iter=3000, verbose=2)
pca.fit(np.vstack([src_embedding, spanish_embeddings]))
print('Variance explained: %.2f' % pca.explained_variance_ratio_.sum())

Variance explained: 0.09


In [ ]:
import matplotlib.pyplot as plt


def plot_similar_word(src_words, src_word2id, src_emb, tgt_words, tgt_word2id, tgt_emb, pca):

    Y = []
    word_labels = []
    for sw in src_words:
        Y.append(src_emb[src_word2id[sw]])
        word_labels.append(sw)
    for tw in tgt_words:
        Y.append(tgt_emb[tgt_word2id[tw]])
        word_labels.append(tw)

    # find tsne coords for 2 dimensions
    Y = pca.transform(Y)
    x_coords = Y[:, 0]
    y_coords = Y[:, 1]

    # display scatter plot
    plt.figure(figsize=(10, 8), dpi=80)
    plt.scatter(x_coords, y_coords, marker='x')

    for k, (label, x, y) in enumerate(zip(word_labels, x_coords, y_coords)):
        color = 'blue' if k < len(src_words) else 'red'  # src words in blue / tgt words in red
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points', fontsize=19,
                     color=color, weight='bold')

    plt.xlim(x_coords.min() - 0.2, x_coords.max() + 0.2)
    plt.ylim(y_coords.min() - 0.2, y_coords.max() + 0.2)
    plt.title('Visualization of the multilingual word embedding space')

    plt.show()
    